In [2]:
#Original code for this cell at https://github.com/theofil/dscout/blob/master/analysis/exampleAnalysis.ipynb
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import subprocess
import bz2
%matplotlib inline

plt.rc('font', size=15)



# data files
path = './data/hiion/' #  The files in this path have been manually processed (removed empty files, and merged a run with two parts into one)
files = os.listdir(path)
files = [f for f in files if 'monitor' not in f]

# getrunNumber from string e.g., int('/data/hiion/scout_326676_000000.monitor.txt'.split('_')[1]
getRun = lambda x: int(x.split('_')[1])



# list to hold dataframes
dfs = []
runIndex= []

for file in files:
    filepath = path+file
    df = pd.read_csv(filepath)
    
  
    # add a column with the run number
    df['run'] = getRun(file) 
    dfs += [df]
    
    #make in index with the run numbers
    runIndex += [getRun(file)]
plt.rc('figure', max_open_warning = 100)




In [3]:
#Fix the runs where the orbit number resets (the warning does not matter)

for df in dfs:
    if not df['orbit'].is_monotonic:
        for i in range(1,len(df)):
            if df['orbit'][i-1]>df['orbit'][i]:
                tmp = i-1
                break
        df['orbit'][tmp+1:] = df['orbit'][tmp] + df['orbit'][tmp+1:]
            

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [4]:
badruns = [0, 1, 4, 8, 9, 11, 28, 30, 33, 34, 35, 36, 39, 46, 47, 50,  57, 65, 68, 69, 70, 72, 77, 78]
goodruns = [i for i in range(len(dfs)) if i not in badruns]

In [5]:
dfg = [] #data frame with only goodruns

for i in range(len(dfs)):
    if i in goodruns : dfg.append(dfs[i])

In [6]:
# Make every df start at 1
for df in dfg:
    df['orbit'] = df['orbit'] - df['orbit'][0] + 1

In [7]:
for j in range(1, len(dfg)):
    dfg[j]['orbit']+=dfg[j-1]['orbit'].tolist()[-1]

In [8]:
merged = pd.concat(dfg, ignore_index=True, sort=False)


In [9]:
# merged.to_csv(path_or_buf="merged.txt", index=False ) 
# # so that someone else can intstant access to the data without running this code

# Two - Leg Muon filtering

In [10]:
def pairing(df, x):
    dfns = df.shift(-x)
    dfps = df.shift(+x)

    logicOS1 = (df.bx == dfns.bx - x) & (df.orbit == dfns.orbit) & (df.charge*dfns.charge < 0)
    OS1 = df[logicOS1].copy()

    logicOS2 = (df.bx == dfps.bx + x) & (df.orbit == dfps.orbit) & (df.charge*dfps.charge < 0)
    OS2 = df[logicOS2].copy()

    OS1 = OS1.reset_index()
    OS2 = OS2.reset_index()

    OS = OS1.merge(OS2, left_on=OS1.index, right_on=OS2.index, suffixes=('_in', '_out'))

    logicSS1 = (df.bx == dfns.bx - x) & (df.orbit == dfns.orbit) & (df.charge*dfns.charge > 0)
    SS1 = df[logicSS1].copy()

    logicSS2 = (df.bx == dfps.bx + x) & (df.orbit == dfps.orbit) & (df.charge*dfps.charge > 0)
    SS2 = df[logicSS2].copy()

    SS1 = SS1.reset_index()
    SS2 = SS2.reset_index()

    SS = SS1.merge(SS2, left_on=SS1.index, right_on=SS2.index, suffixes=('_in', '_out'))

    # slim the dataframes
    def slimDataFrame(df):
        df = df.drop(['run_in', 'index_in','index_out','key_0','orbit_out'], axis=1)
        df = df.rename(columns = {'run_out':'run','orbit_in':'orbit'})
        if False: df = df.set_index(['run','orbit','bx_in'])
        return df

    OS = slimDataFrame(OS)
    SS = slimDataFrame(SS)
    return (OS, SS)

In [11]:
OS, SS = pairing(merged, 1)


In [14]:
len(OS)/len(merged)

0.08033430267706723

In [15]:
OS.head()

,orbit,bx_in,phi_in,eta_in,pt_in,charge_in,bx_out,phi_out,eta_out,pt_out,charge_out,run
0,41208,3458,2.39983,0.000000,4.0,-1,3459,-1.15628,0.80475,3.5,1,326305
1,86938,3481,1.09083,-0.511125,16.5,1,3482,-1.45081,-0.76125,17.5,-1,326305
2,98659,3511,1.12356,0.511125,2.5,1,3512,-2.00713,-0.45675,38.0,-1,326305
3,106772,3458,1.84350,0.511125,139.5,1,3459,-1.02538,0.80475,139.5,-1,326305
4,122121,3476,2.17075,0.804750,3.5,1,3477,-1.61443,0.80475,3.5,-1,326305


In [13]:
OS.to_csv(path_or_buf="twoleg.txt", index=False ) 
# so that someone else can intstant access to the data without running this code